In [1]:
import pandas as pd
import sys
import math
from keras.models import load_model
import tensorflow as tf
from Code.Misc.helper import *
from Code.Misc.features import *
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError
from Code.Misc.split_segments import *
import pyaudio
import wave
import errno
import time
import calendar
import os
from threading import Thread
from collections import deque
import copy
from scipy.io.wavfile import read
import concurrent.futures

# use this backend to display in separate Tk window
#%matplotlib tk
%matplotlib inline

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

C:\Users\walle\anaconda3\envs\TFGPU\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
rem_flag = False
# Clean all old wave files
r = os.listdir('../../Data/Thread_files/')
for i in r:
    try:
        if i.endswith('wav'):
            rem_flag = True
            os.remove(i)
    except OSError as e: # this would be "except OSError, e:" before Python 2.6
        if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
            pass
print(rem_flag)

True


In [3]:
# Recording flag & file count
recording = True
idx = 0

# Initialize pyaudio
p = pyaudio.PyAudio()

# Setup format info
FORMAT = pyaudio.paInt16
CHANNELS = 1                 # single channel for microphone
DEVICE = 1
RATE = 44100
file_len = 5
CHUNK = int(RATE / 2)
print(rem_flag)
# Create fast I/O buffer
d = deque(maxlen=int(RATE / CHUNK * file_len))
# p_name = int(calendar.timegm(time.gmtime()))

True


In [4]:
# set up model
model = load_model("../Misc/Saved_data/Trained_models/2_layer_LSTM.pickle")
cols = ['ang', 'exc', 'neu', 'sad']
df_pred_wav = pd.DataFrame([np.zeros(4)], columns=cols)
data_prev = []
graph_window = 30

In [5]:
# Thread called file I/O function
def write_to_file(arg):
    global wf, idx

    # Write 5s of audio to file
    for elem in arg:
        wf.writeframes(elem)

    # If file is at the desired length close it, rename it to its utc start time (cant get ms?) and open the next temp
    # file for writing to
    if wf.tell() == RATE * file_len:
        wf.close()
        recording_name = str(calendar.timegm(time.gmtime()) - file_len) + '.wav'
        os.rename('../../Data/Thread_files/temp.wav', '../../Data/Thread_files/'+ recording_name)
        wf = wave.open('../../Data/Thread_files/temp.wav', 'wb')
        wf.setnchannels(1)
        wf.setframerate(RATE)
        wf.setsampwidth(2)
        idx += 1
        print ('File created: ' + recording_name)
        return recording_name, idx

In [6]:
executor = concurrent.futures.ThreadPoolExecutor(max_workers=5)
thread_list = []
# Pyaudio callback which appends HW audio buffer data to fast I/O 5s long buffer
def callback(in_data, frame_count, time_info, status):
    if status != 0:
        print ("Non zero status!!!!!!!!!!")
        exit()
    global d
    d.append(in_data)

    # If 5s worth of audio is collected, copy to secondary buffer and pass to thread function for file I/O, then
    # clear 5s buffer
    if len(d) == RATE / CHUNK * file_len:
        frames = copy.copy(d)
        thread_list.append(executor.submit(write_to_file, frames))
        # thread = Thread(target=write_to_file, args=[frames])
        # thread.start()
        d.clear()
        print ('Copied 5s buffer: ')

    return in_data, pyaudio.paContinue

In [7]:
audio_path = "C:/Uni Shit/EECS 4088/Capstone---RTSD-System/Data/CMU_MOSI/Raw/Audio/WAV_16000/Full/"
file_name = "0h-zjBukYpk.wav"

if rem_flag:
    # Open initial temp file and setup
    wf = wave.open('../../Data/Thread_files/temp.wav', 'wb')
    wf.setnchannels(1)
    wf.setframerate(RATE)
    wf.setsampwidth(2)

else:
    wf = wave.open(audio_path + file_name, 'r')
    split_wav = SplitWavAudio(audio_path,file_name)
    split_wav.multiple_split(sec_per_split=5)

#os.remove('../Misc/Saved_data/temp.wav')

In [8]:
# Setup audio input stream
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK,
                input_device_index=DEVICE,
                stream_callback=callback
                )
# Start audio input stream
stream.start_stream()
# time.sleep()
# Capture until we have X number of files for testing
all_result = []
data_int = []
prev_result = None
file_index=0
i=0
thread_path = '../../Data/Thread_files/'
try:
    while recording:
        for thread in concurrent.futures.as_completed(thread_list):
            if file_index < idx:
                file_index += 1
                all_result.extend(thread.result())
                print(all_result[i])
                recorded_file = get_audio(thread_path, all_result[i])
                (nchannels, sampwidth, framerate, nframes, comptype, compname), sample_wav = recorded_file
                data_int = sample_wav
                data_int = np.array(data_int).astype('int16')
                # Generate features from data
                st_features = calculate_features(data_int, RATE, None)
                st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
                # reshape input from (34, 100) to (1, 100, 34)
                st_features = np.array([st_features.T])

                # # predict on model
                with tf.device('/gpu:0'):
                    wav_test_results = model.predict(st_features)
                i += 2
                # continue

            # data_int += list(read(recorded_file))
            # data_int = np.array(data_int).astype('int16')
            # # Generate features from data
            # st_features = calculate_features(data_int, RATE, None)
            # st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
            #
            # # reshape input from (34, 100) to (1, 100, 34)
            # st_features = np.array([st_features.T])
            #
            # # predict on model
            # with tf.device('/gpu:0'):
            #     wav_test_results = model.predict(st_features)
            # if recorded_file:
            #     data_int = []
            #     prev_result = all_result[file_index]
            #     print(prev_result)
            #     print("--" * 40)
            #     print(thread.result())
            #     print("**" * 40)
            #     continue
        # for future in thread_list:
        #     recorded_file = wave.open('../../Data/Thread_files/'+ future.result() + '.wav', 'r')
        #     print(future.result())
        # data_new = wave.readframes(CHUNK)
        # data = read(recorded_file)
        # data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)

        # data_int = []
        # for i in range(d):
        #      data_int = np.append(data_int, data_prev)

        # Generate features from data
        # st_features = calculate_features(data_int, RATE, None)
        # st_features, _ = pad_sequence_into_array(st_features, maxlen=100)

        # reshape input from (34, 100) to (1, 100, 34)
        # st_features = np.array([st_features.T])

        # predict on model
        # with tf.device('/gpu:0'):
        #     wav_test_results = model.predict(st_features)

        # print(wav_test_results.shape)
        if idx > 3:
            recording = False
except:
    if KeyboardInterrupt:
        recording = False

stream.stop_stream()
stream.close()
wf.close()

p.terminate()

Copied 5s buffer: 
File created: 1617239349.wav
1617239349.wav
Copied 5s buffer: 
File created: 1617239354.wav
1617239354.wav
Copied 5s buffer: 
File created: 1617239359.wav
1617239359.wav
Copied 5s buffer: 
File created: 1617239364.wav
1617239364.wav
